# **Deep Learning Models on Solar Panel Data**

By: Jonathan Chartrand & Robert Boutette

Dataset used:
- Ani Kannal, Solar Power Generation Data, Kaggle. Available at: https://www.kaggle.com/datasets/anikannal/solar-power-generation-data.

In [1]:
# Import the required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns                

# Load the data sets
generation_data = pd.read_csv("data/Plant_1_Generation_Data.csv")
weather_data = pd.read_csv("data/Plant_1_Weather_Sensor_Data.csv")

## **Question 0: Dataset Selection and Loading**

We will be analyzing a pair of datasets:

- **Power Generation Data**: ~69,000 rows, 7 columns

- **Weather Sensor Data**: ~3,500 rows, 6 columns

- File format is a basic .csv text file.

### Dataset Description
This dataset includes solar power generation and weather sensor readings collected from two plants in India over a period of 34 days. The data is recorded at 15-minute intervals and consists of power generation at the inverter level and sensor data at the plant level.

The key features of the dataset are:

- **DATE_TIME**: Timestamp of data recording (15-minute intervals)

- **PLANT_ID**: Plant identifier

- **SOURCE_KEY**: Unique inverter identifier

- **DC_POWER**: Direct current power in kW

- **AC_POWER**: Alternating current power in kW

- **DAILY_YIELD**: Cumulative daily power yield in kWh

- **TOTAL_YIELD**: Total inverter output in kWh

- **AMBIENT_TEMPERATURE**: Ambient temperature at the plant (°C)

- **MODULE_TEMPERATURE**: Solar panel temperature (°C)

- **IRRADIATION**: Solar irradiation for the interval (W/m^2)

The purpose of these data sets is to compare a solar generation plant with it's weather data. For the purposes of this assignment we will be analyzing the power/weather data from one plant and have one data frame which includes:

- Plant 1 Generation Data

- Weather Data

As there are two separate csv files, we will have to ensure the datetime format is consistent between the two to match time stamps in the data frame.

In [2]:
# Ensure DATE_TIME in both datasets is in datetime format
generation_data['DATE_TIME'] = pd.to_datetime(generation_data['DATE_TIME'], format='%d-%m-%Y %H:%M', errors='coerce')
weather_data['DATE_TIME'] = pd.to_datetime(weather_data['DATE_TIME'])
# Specify the columns to merge from the weather data
weather_columns = ['DATE_TIME', 'AMBIENT_TEMPERATURE', 'MODULE_TEMPERATURE', 'IRRADIATION']

# Merge the entire generation data with the weather data on the 'DATE_TIME' column
df = pd.merge(generation_data, weather_data[weather_columns], on='DATE_TIME', how='left')


Visualizing the first and last 5 rows of raw data using the *.head()* method we now see:

In [3]:
df.head(df.shape[0])

,DATE_TIME,PLANT_ID,SOURCE_KEY,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION
0,2020-05-15 00:00:00,4135001,1BY6WEcLGh8j5v7,0.0,0.0,0.000,6259559.0,25.184316,22.857507,0.0
1,2020-05-15 00:00:00,4135001,1IF53ai7Xc0U56Y,0.0,0.0,0.000,6183645.0,25.184316,22.857507,0.0
2,2020-05-15 00:00:00,4135001,3PZuoBAID5Wc2HD,0.0,0.0,0.000,6987759.0,25.184316,22.857507,0.0
3,2020-05-15 00:00:00,4135001,7JYdWkrLSPkdwr4,0.0,0.0,0.000,7602960.0,25.184316,22.857507,0.0
4,2020-05-15 00:00:00,4135001,McdE0feGgRqW7Ca,0.0,0.0,0.000,7158964.0,25.184316,22.857507,0.0
...,...,...,...,...,...,...,...,...,...,...
68773,2020-06-17 23:45:00,4135001,uHbuxQJl8lW7ozc,0.0,0.0,5967.000,7287002.0,21.909288,20.427972,0.0
68774,2020-06-17 23:45:00,4135001,wCURE6d3bPkepu2,0.0,0.0,5147.625,7028601.0,21.909288,20.427972,0.0
68775,2020-06-17 23:45:00,4135001,z9Y9gH1T5YWrNuG,0.0,0.0,5819.000,7251204.0,21.909288,20.427972,0.0
68776,2020-06-17 23:45:00,4135001,zBIq5rxdHJRwDNY,0.0,0.0,5817.000,6583369.0,21.909288,20.427972,0.0


We can examine the dataset's datatypes using the .info() method:

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68778 entries, 0 to 68777
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   DATE_TIME            68778 non-null  datetime64[ns]
 1   PLANT_ID             68778 non-null  int64         
 2   SOURCE_KEY           68778 non-null  object        
 3   DC_POWER             68778 non-null  float64       
 4   AC_POWER             68778 non-null  float64       
 5   DAILY_YIELD          68778 non-null  float64       
 6   TOTAL_YIELD          68778 non-null  float64       
 7   AMBIENT_TEMPERATURE  68774 non-null  float64       
 8   MODULE_TEMPERATURE   68774 non-null  float64       
 9   IRRADIATION          68774 non-null  float64       
dtypes: datetime64[ns](1), float64(7), int64(1), object(1)
memory usage: 5.2+ MB


Now we will drop columns 1, 5 and 6 as they are not relevant to any analysis in this assignment:

In [5]:
# Drop the PLANT_ID, DAILY_YIELD and TOTAL_YIELD columns from the frame
df.drop(columns=['PLANT_ID','DAILY_YIELD','TOTAL_YIELD'], inplace=True)
df.head(df.shape[0])

,DATE_TIME,SOURCE_KEY,DC_POWER,AC_POWER,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION
0,2020-05-15 00:00:00,1BY6WEcLGh8j5v7,0.0,0.0,25.184316,22.857507,0.0
1,2020-05-15 00:00:00,1IF53ai7Xc0U56Y,0.0,0.0,25.184316,22.857507,0.0
2,2020-05-15 00:00:00,3PZuoBAID5Wc2HD,0.0,0.0,25.184316,22.857507,0.0
3,2020-05-15 00:00:00,7JYdWkrLSPkdwr4,0.0,0.0,25.184316,22.857507,0.0
4,2020-05-15 00:00:00,McdE0feGgRqW7Ca,0.0,0.0,25.184316,22.857507,0.0
...,...,...,...,...,...,...,...
68773,2020-06-17 23:45:00,uHbuxQJl8lW7ozc,0.0,0.0,21.909288,20.427972,0.0
68774,2020-06-17 23:45:00,wCURE6d3bPkepu2,0.0,0.0,21.909288,20.427972,0.0
68775,2020-06-17 23:45:00,z9Y9gH1T5YWrNuG,0.0,0.0,21.909288,20.427972,0.0
68776,2020-06-17 23:45:00,zBIq5rxdHJRwDNY,0.0,0.0,21.909288,20.427972,0.0


## **Question 1: DNNs for Regression**

For this assignment we will create two different DNN architectures to predict regression of the solar panel output. We will then evaluate their performance and compare the two architectures.

## **Building a Deep Neural Network (DNN) for Regression**

We will now build a Deep Neural Network (DNN) using Keras to predict the DC_POWER of the solar panels. The steps involved are:

1. **Importing necessary libraries**
2. **Preparing the data for training and testing**
3. **Building the DNN model**
4. **Compiling the model**
5. **Training the model**
6. **Evaluating the model**


In [6]:
# Install the required packages

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler



In [7]:
# Split the data into features (X) and target (y)
X = df[['AC_POWER', 'AMBIENT_TEMPERATURE', 'MODULE_TEMPERATURE', 'IRRADIATION']]
y = df['DC_POWER']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the feature data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [8]:
# Build the DNN model
model = Sequential()

# Add input layer and first hidden layer
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))

# Add second hidden layer
model.add(Dense(32, activation='relu'))

# Add third hidden layer
model.add(Dense(16, activation='relu'))

# Add output layer
model.add(Dense(1))



c:\Users\jonni\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


4. **Compiling the model**

In [9]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Print the model summary
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

5. **Training the model**

In [10]:
# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 2s 869us/step - loss: 16382255.0000 - val_loss: 25091474.0000
Epoch 2/50
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 1s 785us/step - loss: 23676580.0000 - val_loss: 17969842.0000
Epoch 3/50
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 1s 783us/step - loss: 17012540.0000 - val_loss: 16389261.0000
Epoch 4/50
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 1s 789us/step - loss: 16183481.0000 - val_loss: 16388743.0000
Epoch 5/50
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 1s 790us/step - loss: 16365878.0000 - val_loss: 16390991.0000
Epoch 6/50
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 1s 787us/step - loss: 16304616.0000 - val_loss: 16388462.0000
Epoch 7/50
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 1s 799us/step - loss: 16426783.0000 - val_loss: 16388309.0000
Epoch 8/50
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 1s 797us/step - loss: 16248342.0000 - val_loss: 16388310.0000
Epoch 9/50
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 1s 774us/step - loss: 16163317.0000 - val_loss: 16388356.0000
Epoch 10/50
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 1s 775us/step - loss:

6. **Evaluating the model**

In [11]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Evaluate the model on the test data
loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')
# Predict the target values for the test set
y_pred = model.predict(X_test)

# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Absolute Error: {mae}')
print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')

430/430 ━━━━━━━━━━━━━━━━━━━━ 0s 581us/step - loss: 15917108.0000
Test Loss: 16184918.0
430/430 ━━━━━━━━━━━━━━━━━━━━ 0s 675us/step
Mean Absolute Error: 3498.8874903111114
Mean Squared Error: 16184917.105533224
R^2 Score: -6.198047959338204e-05


Now we will demonstrate if regularization can help our model. 

In [12]:
from tensorflow.keras.layers import Dropout

# Build the DNN model with regularization
model_reg = Sequential()

# Add input layer and first hidden layer with Dropout
model_reg.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model_reg.add(Dropout(0.2))

# Add second hidden layer with Dropout
model_reg.add(Dense(32, activation='relu'))
model_reg.add(Dropout(0.2))

# Add third hidden layer with Dropout
model_reg.add(Dense(16, activation='relu'))
model_reg.add(Dropout(0.2))

# Add output layer
model_reg.add(Dense(1))

# Compile the model
model_reg.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Print the model summary
model_reg.summary()

# Train the model
history_reg = model_reg.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Evaluate the model on the test data
loss_reg = model_reg.evaluate(X_test, y_test)
print(f'Test Loss with Regularization: {loss_reg}')

# Predict the target values for the test set
y_pred_reg = model_reg.predict(X_test)

# Calculate evaluation metrics
mae_reg = mean_absolute_error(y_test, y_pred_reg)
mse_reg = mean_squared_error(y_test, y_pred_reg)
r2_reg = r2_score(y_test, y_pred_reg)

print(f'Mean Absolute Error with Regularization: {mae_reg}')
print(f'Mean Squared Error with Regularization: {mse_reg}')
print(f'R^2 Score with Regularization: {r2_reg}')

c:\Users\jonni\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 64)             │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 3s 993us/step - loss: 24526578.0000 - val_loss: 24802348.0000
Epoch 2/50
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 1s 916us/step - loss: 23397158.0000 - val_loss: 17031244.0000
Epoch 3/50
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 1s 908us/step - loss: 16739939.0000 - val_loss: 16404490.0000
Epoch 4/50
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 1s 908us/step - loss: 16651086.0000 - val_loss: 16402410.0000
Epoch 5/50
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 1s 926us/step - loss: 16662329.0000 - val_loss: 16415290.0000
Epoch 6/50
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 1s 926us/step - loss: 16759924.0000 - val_loss: 16408986.0000
Epoch 7/50
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 1s 952us/step - loss: 16779442.0000 - val_loss: 16406409.0000
Epoch 8/50
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 1s 969us/step - loss: 16500274.0000 - val_loss: 16424831.0000
Epoch 9/50
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 1s 950us/step - loss: 16640072.0000 - val_loss: 16397493.0000
Epoch 10/50
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 1s 919us/step - loss: